### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트

In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="llama3-70b-8192",  # ✅ 반드시 이 모델명으로
    temperature=0.3,
    api_key=os.getenv("GROQ_API_KEY")
)


In [2]:
from langchain.tools import tool

@tool
def db_search_cafe_func(query: str) -> str:
    """카페 메뉴에서 가격, 재료, 설명 정보를 검색합니다."""
    if "카페라떼" in query:
        return "카페라떼는 ₩5,500이며, 에스프레소와 스팀 밀크로 구성됩니다."
    elif "티라미수" in query:
        return "티라미수는 ₩7,500이며, 마스카포네 치즈, 에스프레소, 코코아 파우더로 구성됩니다."
    else:
        return "해당 항목을 메뉴 DB에서 찾을 수 없습니다."

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 일반적인 음식/음료 정보를 요약합니다."""
    if "라떼" in query:
        return "라떼는 이탈리아에서 유래한 음료로, '라떼'는 이탈리아어로 '우유'를 뜻합니다."
    else:
        return "요청한 항목에 대한 위키 정보를 찾을 수 없습니다."

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 트렌드, 실시간 뉴스 등을 검색합니다."""
    if "콜드브루" in query:
        return "콜드브루는 최근 여름철 인기 음료로 떠오르며 디저트와 함께 소비되는 트렌드가 있습니다."
    return "검색 결과를 찾을 수 없습니다."


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_message = """
당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변
"""

few_shot_examples = """
예시 1:
Human: 아메리카노 정보와 커피 역사를 알려줘
AI: 메뉴 검색과 위키피디아 검색을 진행하겠습니다.
AI: [db_search_cafe_func 도구 호출]
Tool: 아메리카노는 ₩4,000이며, 원두, 물로 구성됩니다. 진한 맛이 특징입니다.
AI: [wiki_summary 도구 호출]
Tool: 커피는 에티오피아에서 기원하여 전 세계로 퍼졌습니다.
AI: 아메리카노는 ₩4,000이며 진한 맛이 특징입니다. 커피는 에티오피아에서 기원하여 유럽을 거쳐 세계 각국에서 사랑받는 음료가 되었습니다.

예시 2:
Human: 콜드브루 요즘 유행하나요?
AI: 웹 검색을 통해 최신 트렌드를 조사하겠습니다.
AI: [tavily_search_func 도구 호출]
Tool: 최근 콜드브루는 다양한 디저트와 함께 소비되는 트렌드가 증가하고 있습니다.
AI: 최근 콜드브루는 다양한 디저트와 함께 소비되는 트렌드가 있으며, 특히 여름철 인기가 많습니다.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message.strip()),
    ("user", few_shot_examples.strip()),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])


In [4]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.3,
    api_key=os.getenv("GROQ_API_KEY")  # 또는 직접 문자열 입력
)

tools = [db_search_cafe_func, wiki_summary, tavily_search_func]
llm_with_tools = llm.bind_tools(tools)

# ✅ 최종 체인
cafe_tool_chain = prompt | llm_with_tools


In [5]:
question = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래도 알려주세요."

response = cafe_tool_chain.invoke({
    "input": question,
    "chat_history": []
})

print("\n✅ 최종 응답:\n")
print(response.content)


BadRequestError: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}